In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models

In [2]:
df = pd.read_csv("export/final_data.csv")
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,{'text': 'deskripsi pekerjaanrequirements si f...,"['electrical inspection', 'management system',..."
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,{'text': 'job role responsible for material av...,"['heavy equipment', 'contract management', 'pr..."
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,{'text': 'internal sales adminjob description ...,"['microsoft office', 'heat exchanger', 'carbon..."
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,{'text': 'job description responsible for hse ...,"['operation management', 'analytical skill', '..."
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,{'text': 'overview our clients is manufacture ...,"['japanese', 'translator', 'english', 'non', '..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17182 entries, 0 to 17181
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       17182 non-null  int64  
 1   job_title                17182 non-null  object 
 2   location                 17182 non-null  object 
 3   salary_currency          17178 non-null  object 
 4   career_level             17182 non-null  object 
 5   experience_level         17182 non-null  object 
 6   education_level          17182 non-null  object 
 7   employment_type          17182 non-null  object 
 8   job_function             17182 non-null  object 
 9   job_benefits             14440 non-null  object 
 10  company_process_time     13377 non-null  object 
 11  company_size             15488 non-null  object 
 12  company_industry         16412 non-null  object 
 13  job_description          17182 non-null  object 
 14  salary                

In [4]:
# Get a NumPy array of all unique values
unique_values_list = df['job_function'].unique()
print(len(unique_values_list))

68


In [5]:
job_function_mapping = {

    # Finance & Accounting
    "Akuntansi / Keuangan,Audit & Pajak": "Finance & Accounting",
    "Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan": "Finance & Accounting",
    "Akuntansi / Keuangan,Perbankan / Jasa Finansial ": "Finance & Accounting",
    "Akuntansi / Keuangan,Keuangan / Investasi Perusahaan ": "Finance & Accounting",
    "Sains,Aktuaria/Statistik": "Finance & Accounting",

    # IT & Software
    "Komputer/Teknologi Informasi,IT-Perangkat Lunak": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Admin Jaringan/Sistem/Database": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Perangkat Keras": "IT & Software",

    # Engineering (Non-IT)
    "Teknik,Teknik Lingkungan": "Engineering",
    "Teknik,Teknik Lainnya": "Engineering",
    "Teknik,Mekanikal": "Engineering",
    "Teknik,Teknik Elektronika": "Engineering",
    "Teknik,Teknik Elektro": "Engineering",
    "Teknik,Teknik Industri": "Engineering",
    "Teknik,Teknik Kimia": "Engineering",
    "Bangunan/Konstruksi,Teknik Sipil/Konstruksi Bangunan": "Engineering",
    "Bangunan/Konstruksi,Survei Kuantitas": "Engineering",
    "Sains,Pertanian": "Engineering",
    "Sains,Teknologi Makanan/Ahli Gizi": "Engineering",
    "Sains,Sains & Teknologi": "Engineering",

    # Manufacturing & Operations
    "Manufaktur,Manufaktur": "Manufacturing & Operations",
    "Manufaktur,Pemeliharaan": "Manufacturing & Operations",
    "Manufaktur,Pembelian/Manajemen Material": "Manufacturing & Operations",
    "Manufaktur,Penjaminan Kualitas / QA": "Manufacturing & Operations",
    "Pelayanan,Logistik/Rantai Pasokan": "Manufacturing & Operations",

    # Sales & Marketing
    "Penjualan / Pemasaran,Pemasaran/Pengembangan Bisnis": "Sales & Marketing",
    "Penjualan / Pemasaran,Digital Marketing": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Jasa Keuangan": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Teknik/Teknikal/IT": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan Ritel": "Sales & Marketing",
    "Penjualan / Pemasaran,Merchandising": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Korporasi": "Sales & Marketing",
    "Penjualan / Pemasaran,Telesales/Telemarketing": "Sales & Marketing",
    "Penjualan / Pemasaran,E-commerce": "Sales & Marketing",
    "Seni/Media/Komunikasi,Periklanan": "Sales & Marketing",

    # HR & Administration
    "Sumber Daya Manusia/Personalia,Sumber Daya Manusia / HR": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Staf / Administrasi umum": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Sekretaris": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Top Management / Manajemen Tingkat Atas": "HR & Administration",
    "Pelayanan,Pengacara / Asisten Legal": "HR & Administration",
    "Pelayanan,Angkatan Bersenjata": "HR & Administration",

    # Creative & Media
    "Seni/Media/Komunikasi,Seni / Desain Kreatif": "Creative & Media",
    "Seni/Media/Komunikasi,Hubungan Masyarakat": "Creative & Media",
    "Seni/Media/Komunikasi,Hiburan": "Creative & Media",
    "Lainnya,Jurnalis/Editor": "Creative & Media",

    # Education & Training
    "Pendidikan/Pelatihan,Pendidikan": "Education",
    "Pendidikan/Pelatihan,Pelatihan & Pengembangan": "Education",

    # Hospitality & Service
    "Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran": "Hospitality & Service",
    "Hotel/Restoran,Hotel/Pariwisata": "Hospitality & Service",
    "Pelayanan,Layanan Pelanggan": "Hospitality & Service",
    "Pelayanan,Teknikal & Bantuan Pelanggan": "Hospitality & Service",
    "Pelayanan,Perawatan Pribadi": "Hospitality & Service",
    "Layanan Kesehatan,Diagnosa/Lainnya": "Hospitality & Service",
    "Layanan Kesehatan,Farmasi": "Hospitality & Service",
    "Layanan Kesehatan,Praktisi/Asisten Medis": "Hospitality & Service",
    "Sains,Bioteknologi": "Hospitality & Service",
    "Sains,Biomedis": "Hospitality & Service",
    "Sains,Kimia": "Hospitality & Service",
    "Sains,Geologi/Geofisika": "Hospitality & Service",

    # Other
    "Bangunan/Konstruksi,Arsitek/Desain Interior": "Other",
    "Bangunan/Konstruksi,Properti/Real Estate": "Other",
    "Lainnya,Pekerjaan Umum": "Other",
    "Lainnya,Lainnya/Kategori tidak tersedia": "Other",
}

df["job_function_group"] = (
    df["job_function"]
    .map(job_function_mapping)
    .fillna("Other")
)

In [6]:
encoders_dict = {}

columns_to_encode = ['job_function_group']
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders_dict[col] = le

df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills,job_function_group
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN,deskripsi pekerjaanrequirements si from reputa...,{'text': 'deskripsi pekerjaanrequirements si f...,"['electrical inspection', 'management system',...",7
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN,job role responsible for material availabili...,{'text': 'job role responsible for material av...,"['heavy equipment', 'contract management', 'pr...",7
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN,internal sales adminjob description we are loo...,{'text': 'internal sales adminjob description ...,"['microsoft office', 'heat exchanger', 'carbon...",9
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN,job description responsible for hse implementa...,{'text': 'job description responsible for hse ...,"['operation management', 'analytical skill', '...",7
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN,overview our clients is manufacture for automo...,{'text': 'overview our clients is manufacture ...,"['japanese', 'translator', 'english', 'non', '...",0


In [7]:
y = df["job_function_group"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts())

n_classes: 10
job_function_group
9    5424
6    2895
3    1929
7    1638
4    1476
2    1302
5     767
0     714
8     599
1     438
Name: count, dtype: int64


In [8]:
RANDOM_STATE = 42
MAX_PER_CLASS = 600

dfs = []

for label, group in df.groupby("job_function_group"):
    if len(group) > MAX_PER_CLASS:
        group = group.sample(
            n=MAX_PER_CLASS,
            random_state=RANDOM_STATE
        )
    dfs.append(group)

df = pd.concat(dfs).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5837 non-null   int64  
 1   job_title                5837 non-null   object 
 2   location                 5837 non-null   object 
 3   salary_currency          5835 non-null   object 
 4   career_level             5837 non-null   object 
 5   experience_level         5837 non-null   object 
 6   education_level          5837 non-null   object 
 7   employment_type          5837 non-null   object 
 8   job_function             5837 non-null   object 
 9   job_benefits             4876 non-null   object 
 10  company_process_time     4531 non-null   object 
 11  company_size             5244 non-null   object 
 12  company_industry         5564 non-null   object 
 13  job_description          5837 non-null   object 
 14  salary                  

In [9]:
tfidf = TfidfVectorizer(
    max_features=1000,        # penting agar RF tidak overload
    ngram_range=(1, 2),
    # stop_words="english",
    min_df=2,
    max_df=0.9,
    sublinear_tf=True
)

In [10]:
df["role_text"] = df["job_description"] + " - " + df['experience_level'] + " - " + df["education_level"]

In [11]:
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,job_description_cleaned,annotations,skills,job_function_group,role_text
0,5894,Photographer,Jakarta Raya,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,Tidak terspesifikasi,Kontrak,"Seni/Media/Komunikasi,Seni / Desain Kreatif","Waktu regular, Senin - Jumat;Kasual (contoh: K...",24 days,501 - 1000 pekerja,Transportasi/Logistik,What is ShipperShipper is a growing technology...,8000000.0,what is shippershipper is a growing technology...,{'text': 'what is shippershipper is a growing ...,"['warehouse management', 'post production', 's...",0,What is ShipperShipper is a growing technology...
1,16927,HEAD CREATIVE,Jakarta Raya,IDR,Supervisor/Koordinator,2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",Asuransi kesehatan;Bisnis (contoh: Kemeja),25 days,201 - 500 pekerja,Kesehatan/Medis,HEAD CREATIVEKualifikasiMemiliki 2 tahun penga...,NaN,head creativekualifikasimemiliki tahun pengala...,{'text': 'head creativekualifikasimemiliki tah...,"['social medium', 'e commerce', 'target market...",0,HEAD CREATIVEKualifikasiMemiliki 2 tahun penga...
2,2243,Japanese Interpreter,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),3 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Lainnya,Jurnalis/Editor","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",5 days,51 - 200 pekerja,Manajemen/Konsulting HR,JAPANESE INTERPRETER (TANGERANG) [50020]COMPAN...,NaN,japanese interpreter tangerang company categor...,{'text': 'japanese interpreter tangerang compa...,"['japanese', 'electrical', 'communication', 'e...",0,JAPANESE INTERPRETER (TANGERANG) [50020]COMPAN...
3,15212,Content Creator,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),3 tahun,Sarjana (S1),Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",24 days,1- 50 pekerja,Makanan & Minuman/Katering/Restoran,CONTENT CREATORLocation : ...,5000000.0,content creatorlocation j...,{'text': 'content creatorlocation jakarta bara...,"['digital marketing', 'social medium', 'adobe ...",0,CONTENT CREATORLocation : ...
4,32356,Graphic Designer,Bali,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,Sarjana (S1),Penuh Waktu,"Seni/Media/Komunikasi,Seni / Desain Kreatif",NaN,NaN,NaN,Olahraga,Responsibilities :Taking design briefs to unde...,3600000.0,responsibilities taking design briefs to under...,{'text': 'responsibilities taking design brief...,"['design brief', 'graphic design', 'adobe phot...",0,Responsibilities :Taking design briefs to unde...


In [12]:
X_text = df["role_text"].astype(str)
y = df["job_function_group"].astype(int)

num_classes = y.nunique()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [14]:
MAX_VOCAB = 30000
MAX_LEN = 200

tokenizer = Tokenizer(
    num_words=MAX_VOCAB,
    oov_token="<OOV>"
)

tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad  = pad_sequences(X_test_seq,  maxlen=MAX_LEN, padding="post")

In [15]:
EMBED_DIM = 256

inputs = layers.Input(shape=(MAX_LEN,))

x = layers.Embedding(
    input_dim=MAX_VOCAB,
    output_dim=EMBED_DIM,
    embeddings_regularizer=tf.keras.regularizers.l2(1e-6)
)(inputs)

# Stacked BiLSTM
x = layers.Bidirectional(
    layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
)(x)

x = layers.Bidirectional(
    layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
)(x)

# Self-Attention
x = layers.Attention()([x, x])

# Dual Pooling
avg_pool = layers.GlobalAveragePooling1D()(x)
max_pool = layers.GlobalMaxPooling1D()(x)
x = layers.Concatenate()([avg_pool, max_pool])

# Classification Head
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 200, 256)  │  7,680,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 200, 256)  │    394,240 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 200, 128)  │    164,352 │ bidirectional[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 200, 128)  │          0 │ bidirectional_1[… │
│ (Attention)         │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ attention[0][0]   │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ concatenate[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     65,792 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 10)        │      1,290 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,339,594 (31.81 MB)

 Trainable params: 8,339,082 (31.81 MB)

 Non-trainable params: 512 (2.00 KB)

In [16]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 52s 818ms/step - accuracy: 0.0971 - loss: 2.3770 - val_accuracy: 0.1381 - val_loss: 2.3077
Epoch 2/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 49s 824ms/step - accuracy: 0.1617 - loss: 2.2813 - val_accuracy: 0.1488 - val_loss: 2.2991
Epoch 3/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 47s 796ms/step - accuracy: 0.2982 - loss: 2.0252 - val_accuracy: 0.4807 - val_loss: 2.1477
Epoch 4/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 49s 832ms/step - accuracy: 0.5331 - loss: 1.3883 - val_accuracy: 0.6049 - val_loss: 1.8914
Epoch 5/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 50s 845ms/step - accuracy: 0.7101 - loss: 0.9034 - val_accuracy: 0.6906 - val_loss: 1.6129
Epoch 6/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 50s 843ms/step - accuracy: 0.8120 - loss: 0.5990 - val_accuracy: 0.7002 - val_loss: 1.3386
Epoch 7/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 51s 858ms/step - accuracy: 0.8584 - loss: 0.4579 - val_accuracy: 0.7248 - val_loss: 1.1183
Epoch 8/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 51s 857ms/step - accuracy: 0.9045 - loss: 0.3162 - val_accu

In [17]:
test_loss, test_acc = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 158ms/step - accuracy: 0.7468 - loss: 0.8891
Test Accuracy: 0.7432


In [18]:
y_pred = model.predict(X_test_pad)
y_pred_label = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_label))

37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 176ms/step 
              precision    recall  f1-score   support

           0       0.80      0.88      0.84       120
           1       0.89      0.84      0.87        88
           2       0.58      0.77      0.66       120
           3       0.91      0.85      0.88       120
           4       0.82      0.74      0.78       120
           5       0.79      0.69      0.74       120
           6       0.88      0.82      0.85       120
           7       0.65      0.67      0.66       120
           8       0.60      0.54      0.57       120
           9       0.63      0.66      0.64       120

    accuracy                           0.74      1168
   macro avg       0.76      0.75      0.75      1168
weighted avg       0.75      0.74      0.74      1168



In [32]:
def predict_job_function(texts):
    seq = tokenizer.texts_to_sequences(texts)
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding="post")
    probs = model.predict(pad)
    labels = probs.argmax(axis=1)
    return labels, probs

In [33]:
persona_0_creative_media = ["""Saya memiliki pengalaman panjang di bidang kreatif dan media, dengan fokus utama pada produksi konten visual dan pengelolaan komunikasi digital. 
Selama beberapa tahun terakhir, saya terbiasa bekerja dengan berbagai format konten seperti desain grafis, video pendek, fotografi, serta penulisan 
konten untuk media sosial dan platform digital lainnya. Saya memahami bagaimana sebuah pesan dapat disampaikan secara efektif melalui visual dan 
narasi yang kuat, sehingga audiens dapat terhubung secara emosional dengan brand atau produk yang ditampilkan. Dalam pekerjaan sehari-hari, saya 
sering terlibat dalam proses brainstorming ide kreatif bersama tim, mulai dari tahap konsep hingga eksekusi akhir. Saya terbiasa mengembangkan 
konsep kampanye, menentukan tone komunikasi, serta memastikan konsistensi identitas visual di berbagai kanal. Selain itu, saya juga terbiasa 
bekerja dengan deadline yang ketat dan melakukan revisi berdasarkan masukan dari stakeholder atau klien. Saya memiliki ketertarikan yang besar 
terhadap tren media digital, perkembangan platform sosial, dan perubahan perilaku audiens. Hal ini mendorong saya untuk terus belajar dan 
menyesuaikan gaya konten agar tetap relevan. Saya terbiasa menggunakan berbagai tools desain dan editing, serta memahami dasar analisis performa 
konten untuk melihat efektivitas kampanye yang dijalankan. Bagi saya, dunia kreatif bukan hanya soal estetika, tetapi juga tentang bagaimana sebuah
ide dapat dikomunikasikan dengan jelas dan berdampak. Saya menikmati proses eksplorasi ide, eksperimen visual, dan kolaborasi lintas fungsi dengan 
tim marketing maupun komunikasi. Saya tertarik untuk terus berkembang di bidang creative dan media, khususnya dalam peran yang memungkinkan saya 
berkontribusi pada pengembangan brand dan storytelling yang kuat.
"""]

persona_1_education = ["""
Saya memiliki latar belakang dan pengalaman panjang di bidang pendidikan, baik dalam konteks pengajaran langsung maupun pengembangan materi 
pembelajaran. Saya terbiasa berinteraksi dengan peserta didik dari berbagai latar belakang, memahami kebutuhan belajar mereka, serta menyesuaikan 
metode pengajaran agar materi dapat dipahami dengan lebih efektif. Saya percaya bahwa proses belajar tidak hanya tentang penyampaian materi, tetapi 
juga tentang membangun pemahaman, motivasi, dan rasa ingin tahu. Dalam peran saya sebagai pendidik, saya sering terlibat dalam penyusunan silabus, 
perencanaan pembelajaran, serta evaluasi hasil belajar. Saya terbiasa menggunakan berbagai pendekatan, baik konvensional maupun berbasis teknologi, 
untuk mendukung proses belajar mengajar. Saya juga memiliki pengalaman dalam melakukan pendampingan dan bimbingan kepada peserta didik yang 
membutuhkan perhatian khusus. Saya memiliki ketertarikan pada pengembangan sistem pendidikan, inovasi metode pembelajaran, serta pemanfaatan 
teknologi untuk meningkatkan kualitas pendidikan. Saya percaya bahwa pendidikan memiliki peran penting dalam membentuk karakter dan kompetensi 
seseorang. Oleh karena itu, saya selalu berusaha menciptakan lingkungan belajar yang kondusif, inklusif, dan mendorong partisipasi aktif. Selain 
mengajar, saya juga terbiasa bekerja secara administratif di lingkungan pendidikan, seperti pengelolaan data akademik dan koordinasi kegiatan 
pembelajaran. Saya memiliki kemampuan komunikasi yang baik, sabar, dan terorganisir. Saya tertarik untuk terus berkontribusi di dunia pendidikan,
baik sebagai pengajar, pengembang kurikulum, maupun peran lain yang mendukung proses belajar.
"""]

persona_2_engineering = ["""
Saya memiliki latar belakang di bidang engineering dan pengalaman bekerja dalam lingkungan teknis yang menuntut ketelitian serta pemahaman sistem
yang kuat. Saya terbiasa menganalisis permasalahan teknis, merancang solusi, serta memastikan implementasi berjalan sesuai dengan spesifikasi dan 
standar yang telah ditetapkan. Dalam pekerjaan saya, aspek keselamatan, efisiensi, dan keandalan selalu menjadi prioritas utama. Saya sering bekerja 
dengan dokumentasi teknis, perhitungan, serta koordinasi dengan berbagai pihak seperti tim operasional dan manajemen proyek. Saya memahami bahwa 
sebuah solusi engineering tidak hanya harus bekerja secara teknis, tetapi juga harus dapat diterapkan secara praktis dan berkelanjutan. Oleh karena
itu, saya terbiasa mempertimbangkan berbagai faktor seperti biaya, waktu, dan risiko dalam setiap keputusan teknis. Saya menikmati proses pemecahan
masalah dan tantangan teknis yang kompleks. Saya terbiasa bekerja secara sistematis dan terstruktur, serta mampu bekerja baik secara mandiri maupun 
dalam tim. Saya juga memiliki ketertarikan untuk terus mempelajari teknologi dan metode baru yang dapat meningkatkan kualitas solusi engineering 
yang saya kerjakan. Bagi saya, bidang engineering adalah tentang mengubah konsep dan teori menjadi solusi nyata yang dapat memberikan manfaat. 
Saya tertarik untuk terus berkembang dalam peran engineering yang memungkinkan saya berkontribusi pada pengembangan sistem, infrastruktur, atau
produk yang andal dan efisien.
"""]

persona_3_finance_accounting = ["""
Saya memiliki pengalaman panjang di bidang keuangan dan akuntansi, dengan fokus pada pengelolaan data keuangan yang akurat dan transparan. 
Saya terbiasa menangani pencatatan transaksi, penyusunan laporan keuangan, serta analisis anggaran. Ketelitian dan konsistensi adalah hal yang 
sangat saya jaga dalam bekerja dengan angka dan data finansial. Dalam keseharian, saya sering bekerja dengan laporan keuangan bulanan dan tahunan,
melakukan rekonsiliasi, serta memastikan kepatuhan terhadap standar akuntansi. Saya memahami pentingnya data keuangan sebagai dasar pengambilan 
keputusan bisnis, sehingga saya selalu berusaha menyajikan informasi yang jelas dan dapat dipercaya. Saya juga terbiasa bekerja dengan berbagai 
pihak, termasuk tim internal dan auditor, untuk memastikan proses keuangan berjalan dengan baik. Saya memiliki kemampuan analisis yang kuat dan 
mampu mengidentifikasi potensi risiko maupun peluang dari sisi keuangan. Selain itu, saya terbiasa menggunakan tools seperti spreadsheet dan sistem
akuntansi untuk mendukung pekerjaan. Saya tertarik untuk terus berkembang di bidang finance dan accounting, khususnya dalam peran yang
memungkinkan saya berkontribusi pada stabilitas dan pertumbuhan organisasi melalui pengelolaan keuangan yang baik.
"""]

persona_4_hr_administration = ["""
Saya memiliki pengalaman di bidang human resources dan administrasi, dengan fokus pada pengelolaan sumber daya manusia dan proses administratif 
perusahaan. Saya terbiasa menangani data karyawan, administrasi kepegawaian, serta mendukung proses rekrutmen dan onboarding. Saya memahami
pentingnya ketertiban administrasi dalam mendukung operasional organisasi. Dalam peran saya, saya sering menjadi penghubung antara karyawan dan 
manajemen, memastikan informasi tersampaikan dengan jelas dan tepat. Saya memiliki kemampuan komunikasi yang baik dan terbiasa menangani dokumen 
serta data dengan tingkat kerahasiaan yang tinggi. Saya juga terbiasa bekerja dengan prosedur dan kebijakan perusahaan. Saya tertarik pada 
pengembangan sistem HR yang efisien dan berorientasi pada kesejahteraan karyawan. Saya percaya bahwa pengelolaan sumber daya manusia yang baik 
dapat menciptakan lingkungan kerja yang produktif dan harmonis. Oleh karena itu, saya selalu berusaha bekerja secara rapi, terstruktur, dan
profesional dalam setiap tugas administrasi yang saya tangani.
"""]

persona_5_hospitality_service = ["""
Saya memiliki pengalaman panjang di bidang hospitality dan layanan pelanggan, dengan fokus pada pemberian pelayanan yang ramah dan profesional. 
Saya terbiasa berinteraksi langsung dengan pelanggan, memahami kebutuhan mereka, serta memastikan pengalaman yang positif selama menggunakan 
layanan. Saya memahami bahwa kepuasan pelanggan merupakan faktor utama dalam industri jasa. Saya sering bekerja dalam lingkungan yang dinamis dan 
menuntut kecepatan serta ketepatan. Saya terbiasa menangani keluhan pelanggan dengan sikap tenang dan solutif. Selain itu, saya juga memahami 
pentingnya kerja sama tim dalam memastikan kelancaran operasional layanan. Saya memiliki kemampuan komunikasi interpersonal yang baik dan mampu 
bekerja dalam sistem shift. Saya menikmati pekerjaan yang melibatkan interaksi dengan banyak orang dan memberikan solusi secara langsung. 
Saya tertarik untuk terus berkembang di bidang hospitality dan service, khususnya dalam peran yang menekankan kualitas layanan dan kepuasan
pelanggan.
"""]

persona_6_it_software = ["""
Saya memiliki latar belakang dan pengalaman di bidang teknologi informasi dan pengembangan perangkat lunak. Saya terbiasa bekerja dengan sistem
berbasis software, memahami alur pengembangan aplikasi, serta memecahkan masalah teknis menggunakan pendekatan logis. Saya memiliki ketertarikan
besar terhadap teknologi dan bagaimana teknologi dapat digunakan untuk meningkatkan efisiensi kerja. Saya sering bekerja dengan data, aplikasi,
dan sistem backend, serta memahami konsep database dan integrasi sistem. Saya terbiasa bekerja secara terstruktur dan mengikuti proses pengembangan 
yang sistematis. Selain itu, saya juga mampu bekerja dalam tim dan berkomunikasi dengan pihak non-teknis untuk menjelaskan solusi yang dikembangkan.
Saya tertarik untuk terus belajar dan mengikuti perkembangan teknologi terbaru. Bagi saya, bidang IT dan software adalah ruang yang terus 
berkembang dan menawarkan tantangan intelektual yang menarik. Saya ingin terus berkontribusi dalam pengembangan solusi digital yang bermanfaat
dan scalable.
"""]

persona_7_manufacturing_operations = ["""
Saya memiliki pengalaman di bidang manufaktur dan operasional, dengan fokus pada pengawasan proses produksi dan efisiensi kerja. 
Saya terbiasa bekerja di lingkungan yang terstruktur dengan prosedur yang jelas. Saya memahami pentingnya kualitas produk, keselamatan kerja, dan 
kelancaran operasional dalam proses manufaktur. Saya sering terlibat dalam pemantauan proses produksi, koordinasi dengan tim, serta memastikan
target operasional tercapai. Saya terbiasa bekerja dengan jadwal dan target yang ketat, serta memahami pentingnya kerja sama tim dalam 
lingkungan produksi. Saya tertarik pada peningkatan efisiensi proses dan optimalisasi operasional. Saya percaya bahwa perbaikan kecil yang 
konsisten dapat memberikan dampak besar pada produktivitas. Saya ingin terus berkembang di bidang manufacturing dan operations dengan fokus 
pada proses yang efektif dan berkelanjutan.
"""]

persona_8_other = ["""
Saya memiliki latar belakang pekerjaan yang bersifat umum dan lintas fungsi. Saya terbiasa menangani berbagai tugas yang beragam, mulai dari
administrasi hingga dukungan operasional. Saya fleksibel dan mudah beradaptasi dengan lingkungan kerja yang berubah. Saya memiliki kemampuan
belajar yang cepat dan siap mendukung berbagai kebutuhan organisasi. Saya terbiasa bekerja dengan berbagai tim dan memahami pentingnya komunikasi 
yang jelas. Saya tertarik pada peran yang memungkinkan saya berkontribusi secara luas dan mendukung kelancaran operasional perusahaan.
"""]

persona_9_sales_marketing = ["""
Saya memiliki pengalaman di bidang sales dan marketing, dengan fokus pada pengembangan pasar dan pencapaian target penjualan. Saya terbiasa 
membangun hubungan dengan pelanggan, memahami kebutuhan mereka, serta menawarkan solusi yang sesuai. Saya memiliki kemampuan komunikasi dan 
negosiasi yang baik. Saya sering terlibat dalam perencanaan strategi pemasaran, pelaksanaan kampanye, serta evaluasi hasil penjualan. Saya memahami
pentingnya analisis pasar dan perilaku pelanggan dalam menentukan strategi yang efektif. Saya tertarik untuk terus berkembang di bidang sales dan 
marketing dengan fokus pada pertumbuhan bisnis dan kepuasan pelanggan.
"""]

In [1]:
labels, probs = predict_job_function(persona_9_sales_marketing)
print(labels)

NameError: name 'predict_job_function' is not defined

In [38]:
# mapping = dict(enumerate(le.classes_))
# mapping

In [39]:
label_map = {0: 'Creative & Media',
 1: 'Education',
 2: 'Engineering',
 3: 'Finance & Accounting',
 4: 'HR & Administration',
 5: 'Hospitality & Service',
 6: 'IT & Software',
 7: 'Manufacturing & Operations',
 8: 'Other',
 9: 'Sales & Marketing'}

print(label_map[labels[0]])

Sales & Marketing


In [24]:
model.save("trained_model.keras")